In [1]:
import pandas as pd
import numpy as np

We first import the databases for each of the three years

In [2]:
data22 = pd.read_csv("base_ano22.csv", sep=",")
data23 = pd.read_csv("base_ano23.csv", sep=",")
data24 = pd.read_csv("base_ano24.csv", sep=",")

/tmp/ipykernel_1611/1633524087.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data22 = pd.read_csv("base_ano22.csv", sep=",")
/tmp/ipykernel_1611/1633524087.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data23 = pd.read_csv("base_ano23.csv", sep=",")


# Data treatment

## 1) Preparing data before merging the dataframes from the years 2022, 2023 and 2024 together

We first remove some duplicated rows 

In [3]:
doublons22 = data22[data22.duplicated(subset=['id_personne', 'annee_soins', 'Mois_soins', 'delta'], keep=False)]
doublons23 = data23[data23.duplicated(subset=['id_personne', 'annee_soins', 'Mois_soins', 'delta'], keep=False)]
doublons24 = data24[data24.duplicated(subset=['id_personne', 'annee_soins', 'Mois_soins', 'delta'], keep=False)]

In [4]:
results22 = doublons22.loc[doublons22.groupby(['id_personne', 'annee_soins', 'Mois_soins'])['sum_pres'].idxmax()]
results23 = doublons23.loc[doublons23.groupby(['id_personne', 'annee_soins', 'Mois_soins'])['sum_pres'].idxmax()]
results24 = doublons24.loc[doublons24.groupby(['id_personne', 'annee_soins', 'Mois_soins'])['sum_pres'].idxmax()]

In [5]:
index_to_drop22 = list(set(doublons22.index) - set(results22.index))
data22 = data22.drop(index_to_drop22)
index_to_drop23 = list(set(doublons23.index) - set(results23.index))
data23 = data23.drop(index_to_drop23)
index_to_drop24 = list(set(doublons24.index) - set(results24.index))
data24 = data24.drop(index_to_drop24)

We create 2 new variables : ra is the average refund and fra is the average amount left to pay to the client 

In [6]:
data22['ra'] = np.where(data22['nb_acte'] != 0, data22['remb_mutuelle'] / data22['nb_acte'], 0)
data23['ra'] = np.where(data23['nb_acte'] != 0, data23['remb_mutuelle'] / data23['nb_acte'], 0)
data24['ra'] = np.where(data24['nb_acte'] != 0, data24['remb_mutuelle'] / data24['nb_acte'], 0)

data22['fra'] = np.where(data22['nb_acte'] != 0, data22['frais_reels'] / data22['nb_acte'], 0)
data23['fra'] = np.where(data23['nb_acte'] != 0, data23['frais_reels'] / data23['nb_acte'], 0)
data24['fra'] = np.where(data24['nb_acte'] != 0, data24['frais_reels'] / data24['nb_acte'], 0)

Wa create 2 new variables : ra is the average refund and fra is the average amount left to pay to the client

In [7]:
data22['ra'] = np.where(data22['nb_acte'] != 0, data22['remb_mutuelle'] / data22['nb_acte'], 0)
data23['ra'] = np.where(data23['nb_acte'] != 0, data23['remb_mutuelle'] / data23['nb_acte'], 0)
data24['ra'] = np.where(data24['nb_acte'] != 0, data24['remb_mutuelle'] / data24['nb_acte'], 0)

data22['fra'] = np.where(data22['nb_acte'] != 0, data22['frais_reels'] / data22['nb_acte'], 0)
data23['fra'] = np.where(data23['nb_acte'] != 0, data23['frais_reels'] / data23['nb_acte'], 0)
data24['fra'] = np.where(data24['nb_acte'] != 0, data24['frais_reels'] / data24['nb_acte'], 0)

We create new columns to get the amount paid and perceived from the insurance for each month, we then do this for the "ra" and "fra" variables. In our new table, there will be one column that gives the value of these two variables for each month.

In [8]:
for mois in range(1, 13):
    mois_str = str(mois).zfill(2)
    nom_variable = 'ra_' + mois_str + '22' 
    data22[nom_variable] = data22.loc[data22['Mois_soins'] == float(mois), 'ra']

for mois in range(1, 13):
    mois_str = str(mois).zfill(2) 
    nom_variable = 'ra_' + mois_str + '23' 
    data23[nom_variable] = data23.loc[data23['Mois_soins'] == float(mois), 'ra']

for mois in range(1, 9):
    mois_str = str(mois).zfill(2) 
    nom_variable = 'ra_' + mois_str + '24' 
    data24[nom_variable] = data24.loc[data24['Mois_soins'] == float(mois), 'ra']

In [9]:
for mois in range(1, 13):
    mois_str = str(mois).zfill(2)
    nom_variable = 'fra_' + mois_str + '22' 
    data22[nom_variable] = data22.loc[data22['Mois_soins'] == float(mois), 'fra']

for mois in range(1, 13):
    mois_str = str(mois).zfill(2) 
    nom_variable = 'fra_' + mois_str + '23' 
    data23[nom_variable] = data23.loc[data23['Mois_soins'] == float(mois), 'fra']

for mois in range(1, 9):
    mois_str = str(mois).zfill(2) 
    nom_variable = 'fra_' + mois_str + '24' 
    data24[nom_variable] = data24.loc[data24['Mois_soins'] == float(mois), 'fra']

We now aggregate each dataframe on the 'id_personne' column to then merge them together on this column. We use the criteria first and max for redundant variables and sum to get the total amount refunded and paid for each month by each client.

In [11]:
data22_agg= data22.groupby('id_personne').agg({
    'entite_eco': 'first', 
    'personne_morale': 'first',
    'annee_soins': 'first',
    'colloc': 'first',
    'adh_fac':'first', 
    'type_cont':'first',
    'genre': 'first',
    'code_postal': 'first',
    'Age22': 'first',
    'type_benef': 'first',
    'tranche_age22': 'first',
    'Foyer22': 'first',
    'code_postal': 'first',
    'rg_benef': 'first',
    'PRES2201': 'max',
    'PRES2202': 'max',
    'PRES2203': 'max', 
    'PRES2204': 'max',
    'PRES2205': 'max',
    'PRES2206': 'max',
    'PRES2207': 'max',
    'PRES2208': 'max',
    'PRES2209': 'max',
    'PRES2210': 'max',
    'PRES2211': 'max',
    'PRES2212': 'max',
    'ra_0122': 'sum', 
    'ra_0222': 'sum',
    'ra_0322': 'sum',
    'ra_0422': 'sum', 
    'ra_0522': 'sum', 
    'ra_0622': 'sum',
    'ra_0722': 'sum', 
    'ra_0822': 'sum', 
    'ra_0922': 'sum',
    'ra_1022': 'sum', 
    'ra_1122': 'sum', 
    'ra_1222': 'sum',
    'fra_0122': 'sum', 
    'fra_0222': 'sum', 
    'fra_0322': 'sum',
    'fra_0422': 'sum', 
    'fra_0522': 'sum',
    'fra_0622': 'sum',
    'fra_0722': 'sum', 
    'fra_0822': 'sum', 
    'fra_0922': 'sum',
    'fra_1022': 'sum', 
    'fra_1122': 'sum', 
    'fra_1222': 'sum',
})

In [12]:
data23_agg= data23.groupby('id_personne').agg({
    'entite_eco': 'first', 
    'personne_morale': 'first',
    'annee_soins': 'first',
    'colloc': 'first',
    'adh_fac':'first', 
    'type_cont':'first',
    'genre': 'first',
    'code_postal': 'first',
    'Age23': 'first',
    'type_benef': 'first',
    'tranche_age23': 'first',
    'Foyer23': 'first',
    'code_postal': 'first',
    'rg_benef': 'first',
    'PRES2301': 'max',
    'PRES2302': 'max',
    'PRES2303': 'max', 
    'PRES2304': 'max',
    'PRES2305': 'max',
    'PRES2306': 'max',
    'PRES2307': 'max',
    'PRES2308': 'max',
    'PRES2309': 'max',
    'PRES2310': 'max',
    'PRES2311': 'max',
    'PRES2312': 'max',
    'ra_0123': 'sum', 
    'ra_0223': 'sum',
    'ra_0323': 'sum',
    'ra_0423': 'sum', 
    'ra_0523': 'sum', 
    'ra_0623': 'sum',
    'ra_0723': 'sum', 
    'ra_0823': 'sum', 
    'ra_0923': 'sum',
    'ra_1023': 'sum', 
    'ra_1123': 'sum', 
    'ra_1223': 'sum',
    'fra_0123': 'sum', 
    'fra_0223': 'sum', 
    'fra_0323': 'sum',
    'fra_0423': 'sum', 
    'fra_0523': 'sum',
    'fra_0623': 'sum',
    'fra_0723': 'sum', 
    'fra_0823': 'sum', 
    'fra_0923': 'sum',
    'fra_1023': 'sum', 
    'fra_1123': 'sum', 
    'fra_1223': 'sum',
})

In [13]:
data24_agg= data24.groupby('id_personne').agg({
    'entite_eco': 'first', 
    'personne_morale': 'first',
    'annee_soins': 'first',
    'colloc': 'first',
    'adh_fac':'first', 
    'type_cont':'first',
    'genre': 'first',
    'code_postal': 'first',
    'Age24': 'first',
    'type_benef': 'first',
    'tranche_age23': 'first',
    'Foyer24': 'first',
    'code_postal': 'first',
    'rg_benef': 'first',
    'PRES2401': 'max',
    'PRES2402': 'max',
    'PRES2403': 'max', 
    'PRES2404': 'max',
    'PRES2405': 'max',
    'PRES2406': 'max',
    'PRES2407': 'max',
    'PRES2408': 'max',
    'PRES2409': 'max',
    'PRES2410': 'max',
    'PRES2411': 'max',
    'PRES2412': 'max',
    'ra_0124': 'sum', 
    'ra_0224': 'sum',
    'ra_0324': 'sum',
    'ra_0424': 'sum', 
    'ra_0524': 'sum', 
    'ra_0624': 'sum',
    'ra_0724': 'sum', 
    'ra_0824': 'sum',  
    'fra_0224': 'sum', 
    'fra_0324': 'sum',
    'fra_0424': 'sum', 
    'fra_0524': 'sum',
    'fra_0624': 'sum',
    'fra_0724': 'sum', 
    'fra_0824': 'sum',
})

Before merging we replace all the Nans in the ra and fra columns by zero to make the difference between people who did not perceive any money from the insurance and those who where not attached to it after the merge

In [14]:
data22_agg.fillna(0, inplace=True)
data23_agg.fillna(0, inplace=True)
data24_agg.fillna(0, inplace=True)

## 2) Merging the dataframes 

We new proceed to merge the three dataframes while also removing the duplicate columns creating by the merge procedure

In [15]:
data22_23_merged = pd.merge(data22_agg, data23_agg, on='id_personne', how='outer', suffixes=('', '_y'))

We also need to make sure that values that only appear in the duplicate columns of data23_agg are updated in the merged dataframe before removing these columns. We also do this operation on other columns that will not appear as duplicates but that we do not need for each year ('Foyer22', 'tranche_age22', 'Age22')

In [16]:
for col in data22_agg.columns:
    if col != 'id_personne' and col + '_y' in data22_23_merged.columns:
        data22_23_merged[col] = data22_23_merged[col].combine_first(data22_23_merged[col + '_y'])

In [17]:
data22_23_merged['Foyer22'] = data22_23_merged['Foyer22'].combine_first(data22_23_merged['Foyer23'])
data22_23_merged['tranche_age22'] = data22_23_merged['tranche_age22'].combine_first(data22_23_merged['tranche_age23'])
data22_23_merged['Age22'] = data22_23_merged['Age22'].combine_first(data22_23_merged['Age23'])

In [18]:
data22_23_merged.drop(data22_23_merged.filter(regex='_y$').columns, axis=1, inplace=True)
data22_23_merged.drop(columns=['Foyer23', 'tranche_age23', 'Age23'], inplace=True)

We now merge with the 2024 data following the same procedure

In [19]:
full_data = pd.merge(data22_23_merged, data24_agg, on='id_personne', how='outer', suffixes=('', '_y'))

In [20]:
for col in data22_23_merged.columns:
    if col != 'id_personne' and col + '_y' in full_data.columns:
        full_data[col] = full_data[col].combine_first(full_data[col + '_y'])

In [21]:
full_data['Foyer22'] = full_data['Foyer22'].combine_first(full_data['Foyer24'])
full_data['tranche_age22'] = full_data['tranche_age22'].combine_first(full_data['tranche_age23'])
full_data['Age22'] = full_data['Age22'].combine_first(full_data['Age24'])

In [22]:
full_data.drop(full_data.filter(regex='_y$').columns, axis=1, inplace=True)
full_data.drop(columns=['Foyer24', 'tranche_age23', 'Age24'], inplace=True)

We now have the final dataframe we will use for clustering 

In [23]:
full_data.head()

,entite_eco,personne_morale,annee_soins,colloc,adh_fac,type_cont,genre,code_postal,Age22,type_benef,...,ra_0624,ra_0724,ra_0824,fra_0224,fra_0324,fra_0424,fra_0524,fra_0624,fra_0724,fra_0824
id_personne,,,,,,,,,,,,,,,,,,,,,
0000031a283216ec643bea5b46511325bb088033d75a379188a37162cf4ebb10,EE7,PM7,2022.0,non,non,0.0,Femme,13540.0,45.0,Salarié,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00002650c3250e7fe5b197f45a27ab2e4fc38c1cdee65ce8e08a43186b8ac4b8,EE2703,PM3563,2022.0,non,non,0.0,Enfant,41000.0,10.0,Enfant mineur,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0000329a0a7e29081ee5dafa006d627f7eb4db82c046ad2f36f609ccd3101a63,EE1979,PM2,2022.0,0,0,0.0,Enfant,99999.0,3.0,Enfant mineur,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00003cd2efe009e8bfea54a7de13f162e5359495da8cf9f04258104c845ffdfe,EE2948,PM3908,2022.0,non,non,0.0,Enfant,75018.0,26.0,Enfant majeur,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0000458142f363a32276d020258f637fa4b173c8471e6e73049eadea49753db1,EE2149,PM2680,2022.0,non,non,0.0,Homme,33400.0,32.0,Salarié,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Finally, we save the dataframe and we will reload it in the kmeans.ipynb file

In [24]:
full_data.to_csv('full_data.csv', index=False)